In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.dates import DateFormatter

In [2]:
# Update rcParams to set the default font to Times New Roman
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'DejaVu Serif'

In [3]:
def load_systems_data(degradation_setting, rain_scenario, file_type="concentrations"):
    if file_type=="concentrations":
        df_res = pd.read_csv(os.path.join("systems_output", "preprocessed", f"{degradation_setting}_{rain_scenario}.csv"))
        df_res = df_res.loc[df_res.minutes>0,:]
    elif file_type=="flow_rates":
        df_res = pd.read_csv(os.path.join("systems_output", "preprocessed", f"flow_rates_{degradation_setting}_{rain_scenario}.csv"))
        df_res.minutes = df_res.minutes - 70 # ensure time consistency with concentrations
        df_res = df_res.loc[df_res.minutes>0,:]
    else:
        assert False, f"file_type {file_type} not supported"

    df_res.time = pd.to_datetime(df_res.time)

    # drop MW064, RW156 (invalid calculations of ++systems)
    df_res = df_res.loc[~df_res.sampling_point.isin(["MW064", "RW156"]), :]
    # rename sampling locations according to naming in the paper
    df_res.sampling_point = df_res.sampling_point.map({"MW022": "1", "MW023": "2", "MW017": "3", "MW043": "4", "MW048": "5", 
                        "RW157": "6", "MW046": "7", "MW061": "8", "RW143": "9", "RW141": "10",
                        "RW155": "11", "MW059": "12", "RW211": "13", "MW054": "14",
                        "RW126": "15", "MW052": "16"})
    return df_res

In [27]:
rain_scenarios = ["KeinRegen", "Nieselregen", "MittelstarkerRegen"]
degradation_setting = "no_degradation"
df_1 = load_systems_data(degradation_setting, rain_scenarios[0], file_type="flow_rates")
df_2 = load_systems_data(degradation_setting, rain_scenarios[1], file_type="flow_rates")
df_3 = load_systems_data(degradation_setting, rain_scenarios[2], file_type="flow_rates")

In [ ]:
df_1.describe()

,minutes,l/s,memilio_id,time
count,1.427659e+06,1.427659e+06,1.427659e+06,1427659
mean,9.977528e+03,1.724454e-01,1.220218e+02,2024-01-07 23:27:31.706437376
min,1.000000e+00,1.600100e-01,1.000000e+00,2024-01-01 01:11:00
25%,4.934000e+03,1.694000e-01,5.800000e+01,2024-01-04 11:24:00
50%,1.010900e+04,1.734400e-01,1.190000e+02,2024-01-08 01:39:00
75%,1.505400e+04,1.763800e-01,1.860000e+02,2024-01-11 12:04:00
max,2.016400e+04,1.789900e-01,2.500000e+02,2024-01-15 01:14:00
std,5.809567e+03,4.683059e-03,7.253636e+01,NaN


In [ ]:
# delete extreme values at the beginning and end
df_1 = df_1.loc[df_1["l/s"]<0.179,:]
df_1 = df_1.loc[df_1["l/s"]>0.163,:]

In [ ]:
df_1["rain_scenario"] = rain_scenarios[0]
df_2["rain_scenario"] = rain_scenarios[1]
df_3["rain_scenario"] = rain_scenarios[2]
df_rain = pd.concat([df_1, df_2, df_3])

In [ ]:
del df_1, df_2, df_3

In [ ]:
date_form = DateFormatter("%-d")


def rain_scenarios_next_to_each_other(station):
    df_station = df_rain.loc[df_rain.sampling_point==station,:].sort_values("time")

    fig, axs = plt.subplots(ncols=3, sharey=False, figsize=(18, 6), dpi=300)
    for i, (variate, variate_title) in enumerate(zip(rain_scenarios, ["no precipitation", "moderate gentle rain showers", "moderate rain showers"])):
        sns.lineplot(data=df_station.loc[df_station.rain_scenario==variate,:], x="time", y="l/s", ax=axs[i], color=["slategray", "cornflowerblue", "navy"][i])
        axs[i].set_title(f"{variate_title}", fontsize=20)    
        axs[i].xaxis.set_major_formatter(date_form) 
        axs[i].set_xlabel(f"simulation time (days)", fontsize=15)
        if i==0:
            axs[i].set_ylabel(f"l/s", fontsize=15)
        else:
            axs[i].set_ylabel("")
        axs[i].set_xticklabels(["0", "2", "4", "6", "8", "10", "12", "14"])
        axs[i].tick_params(axis='x', labelsize=12)
        axs[i].tick_params(axis='y', labelsize=12)

    axs[0].set_ylim(0.0, 0.3)
    axs[1].set_ylim(0.0, 100)
    #axs[2].set_ylim(0.16, 0.179)

    fig.savefig(os.path.join("systems_output", "plots", "station_concentration", "rain_scenarios", f"flow_rates_{degradation_setting}_{station}.png"))
    plt.close(fig)

for station in ["16"]: #, "16"
    rain_scenarios_next_to_each_other(station)

/tmp/ipykernel_13562/3507862855.py:17: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[i].set_xticklabels(["0", "2", "4", "6", "8", "10", "12", "14"])
/tmp/ipykernel_13562/3507862855.py:17: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[i].set_xticklabels(["0", "2", "4", "6", "8", "10", "12", "14"])
/tmp/ipykernel_13562/3507862855.py:17: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[i].set_xticklabels(["0", "2", "4", "6", "8", "10", "12", "14"])
